In [46]:
import pandas as pd 
import re
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import spacy
from gensim.models import Word2Vec
import multiprocessing
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import logging
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/derricklewis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
df = pd.read_csv("./2000/2000_text_per_user.csv", index_col=0 )

In [6]:
#test_df = df.iloc[:200,:]

In [49]:
def makedict():
    d ={}
    i = 0
    for item in df['id'].values:
        if item not in d.keys():
            d[item] = []
    for index, row in df.iterrows():
        cpc = row['all_text'].replace("\\n", " ").replace('\\'," ")
        cpc = re.sub('u[1-9]....', '', cpc)
        cpc = cpc.lower()
        cpc = re.findall(r"[\w']+", cpc)
        cpc = list(filter(lambda x:x not in list(stopwords.words('english')),cpc))
        cpc = ' '.join(cpc)
        d[row['id']].append(cpc)
    for item in d.keys():
        d[item] = ' '.join(d[item])
    return d

In [50]:
def makesentences():
    d =[]
    i = 1
    for index, row in df.iterrows():
        cpc = row['all_text'].replace("\\n", " ").replace('\\'," ")
        cpc = re.sub('u[1-9]....', '', cpc)
        cpc = cpc.lower()
        cpc = re.findall(r"[\w']+", cpc)
        d.append(cpc)
    return d

In [51]:
sentences = makesentences()

In [53]:
textdict = makedict()

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle

with open('./2000/2000_data.p', 'wb') as fp:
    pickle.dump(textdict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./2000/2000_data.p', 'rb') as fp:
    data = pickle.load(fp)

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data.values())

In [ ]:
tsvd = TruncatedSVD(n_components=3)

In [ ]:
X_s = tsvd.fit_transform(X)

In [ ]:
tsvd.singular_values_

----

In [ ]:
kmeans = KMeans(n_clusters=18,verbose=20).fit(X_s)

In [ ]:
fig = plt.figure(figsize=(16,10))
ax1 = fig.add_subplot(111, projection='3d')

x = X_s[:,0]
y = X_s[:,1]
z = X_s[:,2]

ax1.scatter(x,y,z, marker=".", c=kmeans.labels_, s=30)
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
x = X_s[:,0]
y = X_s[:,1]
ax.scatter(x,y, marker=".", c=kmeans.labels_, s=30)
plt.show()

---

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X = TfidfVectorizer().fit_transform(data.values())

In [ ]:
tsvd = TruncatedSVD(n_components=3)

In [ ]:
X_s = tsvd.fit_transform(X)

In [ ]:
kmeans = KMeans(n_clusters=18,verbose=20).fit(X_s)

In [ ]:
fig = plt.figure(figsize=(16,10))
ax1 = fig.add_subplot(111, projection='3d')

x = X_s[:,0]
y = X_s[:,1]
z = X_s[:,2]

ax1.scatter(x,y,z, marker=".", c=kmeans.labels_, s=30)
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
x = X_s[:,0]
y = X_s[:,1]
ax.scatter(x,y, marker=".", c=kmeans.labels_, s=30)
plt.show()

---

In [ ]:
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
# cores = multiprocessing.cpu_count()

In [ ]:
model = Word2Vec(min_count=1,workers=8,window=4,size=300,sample=6e-5)

In [ ]:
model.build_vocab(sentences,progress_per=10000)

In [ ]:
model.train(sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

In [ ]:
model.wv.save_word2vec_format('modell.txt')

In [ ]:
del data['16758']

In [ ]:
# model.wv.most_similar(positive=["lol"])

In [ ]:
# model.wv.most_similar(positive=["lunch", "dinner"], negative=["second"], topn=3)    

In [ ]:
!python -m spacy init-model en /tmp/custom_vectors --vectors-loc modell.txt

In [ ]:
nlp = spacy.load("/tmp/custom_vectors")

In [ ]:
for stopword in list(stopwords.words('english')):
    nlp.vocab[stopword].is_stop = True

In [ ]:
def maketokens():
    xl = [value for value in data.values()]
    xk = [key for key in data.keys()]
    x = {}
    i = 0
    for index,text in enumerate(xl):
        x[xk[index]]=nlp(text)
    return x

In [ ]:
tokens = maketokens()

In [ ]:
def make_similarity_index():
    xl = [value for value in tokens.values()]
    xk = [key for key in tokens.keys()]
    sim_index = {}
    i = 1
    for index, token in enumerate(xl):
        sim_token = []
        for doc in xl:
            sim_token.append(token.similarity(doc))
            
        sim_index[xk[index]] = sim_token
        i+=1
        print(i, 'user: ', xk[index])
    return sim_index


In [45]:
sim_index = make_similarity_index()

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201


In [41]:
sim_index

{'8': [1.0,
  0.2743105259967317,
  0.23729751777325128,
  0.20222496923260364,
  0.2088714212099206,
  0.19977196620660592,
  0.47012866791056196,
  0.1397643178308734,
  0.22551945567087708,
  0.10293746039175013,
  0.22758493738344934,
  0.3962376608852453,
  0.2889435313996061,
  0.3015616221692181,
  0.29248808774922,
  0.0,
  0.17490281318677411,
  0.43817446913682173,
  0.18674098882793244,
  0.20154039124694512,
  0.23404835538265858,
  0.3762285501357099,
  0.31707275482336394,
  0.2135175450126056,
  0.0,
  0.2102799851062087,
  0.35794945551983504,
  0.33662105972682205,
  0.29549043899536354,
  0.07267727671272736,
  0.16045328069465326,
  0.2515844498598056,
  0.11090781059492032,
  0.2787871634804326,
  0.20124975083727745,
  0.20699253190258088,
  0.2567701692433895,
  0.2079608001109133,
  0.5016570659426097,
  0.23878054481745842,
  0.05710325366796635,
  0.2027986548745071,
  0.19216641208496146,
  0.08237463597840096,
  0.22993255047231256,
  0.19335686861033102,
  0

In [ ]:
sim_columns = [i for i in sim_index.values()]
sim_column_names = [i for i in sim_index.keys()]

In [ ]:
df2 = pd.DataFrame(sim_columns,columns = sim_column_names)

In [ ]:
df2.index = sim_column_names

In [ ]:
df2.head()

In [ ]:
df2.to_csv('2000_similarity_index.csv')

---

In [ ]:
pca = PCA(n_components = 3)

In [ ]:
X = pca.fit_transform(df2)

In [ ]:
pca.singular_values_

In [ ]:
kmeans = KMeans(n_clusters=18,n_jobs=1).fit(df2)

In [ ]:
kmeans.labels_

---

In [ ]:
fig = plt.figure(figsize=(16,10))
ax1 = fig.add_subplot(111, projection='3d')

x = X[:,0]
y = X[:,1]
z = X[:,2]

ax1.scatter(x,y,z, marker=".", c=kmeans.labels_, s=30)
plt.show()


In [ ]:
fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
x = X[:,0]
y = X[:,1]
ax.scatter(x,y, marker=".", c=kmeans.labels_, s=30)
plt.show()